<a href="https://colab.research.google.com/github/pedro07br/IA_Image_preview_test/blob/main/IA_Image_preview_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import MobileNetV2
import numpy as np
from tensorflow.keras.preprocessing import image
from google.colab import files
import zipfile
import os

uploaded = files.upload()  # selecione sem_classificacao.zip

zip_path = "/content/sem_classificacao.zip"  # novo nome do arquivo
extract_path = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Arquivo extraído para:", extract_path)


dataset_path = "/content/dataset/sem_classificacao"  # assumindo que a pasta interna do ZIP se chama sem_classificacao

# ---------- 4️⃣ Classes ----------
class_names = [
    'abacaxi','banana', 'batata', 'batata_doce', 'cebola',
    'goiaba','laranja', 'limao', 'maca','morango','pepino',
    'tomate', 'uva'
]

Saving sem_classificacao.zip to sem_classificacao.zip
✅ Arquivo extraído para: /content/dataset


In [4]:
# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224,224),
    batch_size=16,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224,224),
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

# Transfer Learning: MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False  # Congelar base no início

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(class_names), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Found 1061 images belonging to 13 classes.
Found 261 images belonging to 13 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,423,629 (9.25 MB)

 Trainable params: 165,645 (647.05 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [5]:
import math

steps_per_epoch = math.ceil(train_generator.samples / train_generator.batch_size)
validation_steps = math.ceil(validation_generator.samples / validation_generator.batch_size)

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    epochs=15
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
67/67 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - accuracy: 0.5167 - loss: 1.7146 - val_accuracy: 0.7931 - val_loss: 0.5891
Epoch 2/15
67/67 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.8847 - loss: 0.4133 - val_accuracy: 0.8851 - val_loss: 0.2887
Epoch 3/15
67/67 ━━━━━━━━━━━━━━━━━━━━ 81s 993ms/step - accuracy: 0.9267 - loss: 0.2487 - val_accuracy: 0.9042 - val_loss: 0.2814
Epoch 4/15
67/67 ━━━━━━━━━━━━━━━━━━━━ 83s 1s/step - accuracy: 0.9399 - loss: 0.1998 - val_accuracy: 0.8774 - val_loss: 0.3759
Epoch 5/15
67/67 ━━━━━━━━━━━━━━━━━━━━ 89s 1s/step - accuracy: 0.9519 - loss: 0.1963 - val_accuracy: 0.8774 - val_loss: 0.4440
Epoch 6/15
67/67 ━━━━━━━━━━━━━━━━━━━━ 66s 989ms/step - accuracy: 0.9492 - loss: 0.1417 - val_accuracy: 0.8812 - val_loss: 0.3720
Epoch 7/15
67/67 ━━━━━━━━━━━━━━━━━━━━ 66s 988ms/step - accuracy: 0.9636 - loss: 0.1291 - val_accuracy: 0.8697 - val_loss: 0.3881
Epoch 8/15
67/67 ━━━━━━━━━━━━━━━━━━━━ 81s 971ms/step - accuracy: 0.9769 - loss: 0.0904 - val_accuracy: 0.8812

In [13]:
uploaded = files.upload()

# Função de previsão
def prever_imagem(caminho_imagem):
    img = image.load_img(caminho_imagem, target_size=(224,224))
    img_array = image.img_to_array(img)/255.0
    img_array = np.expand_dims(img_array, axis=0)

    pred = model.predict(img_array)
    classes = list(train_generator.class_indices.keys())
    print("Classe prevista:", classes[np.argmax(pred)])
    return classes[np.argmax(pred)]

# Iterar sobre todas as imagens enviadas
for filename in uploaded.keys():
    caminho = "/content/" + filename
    print("\nImagem:", filename)
    alimento = prever_imagem(caminho)


Saving Captura de tela 2025-11-04 165522.png to Captura de tela 2025-11-04 165522.png

Imagem: Captura de tela 2025-11-04 165522.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Classe prevista: banana
